<a href="https://colab.research.google.com/github/rii92/Datmin/blob/main/other-package/scrapping_waktu_video_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install beautifulsoup4 requests python-dateutil

In [ ]:
# video_urls = [
#     "https://youtu.be/0Egm9jHGoVg",
#     "https://youtu.be/iYJXOPTgLig",
#     # Tambahkan URL video lainnya
# ]

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# Fungsi untuk mengambil durasi video dari URL
def get_video_duration(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Cari tag meta dengan properti "og:duration"
        duration_tag = soup.find('meta', {'itemprop': 'duration'})
        if duration_tag:
            duration_iso = duration_tag['content']
            return parse_duration_to_seconds(duration_iso)
        else:
            # Alternatif: Cari tag script yang mengandung "approxDurationMs"
            for script in soup.find_all('script'):
                if 'approxDurationMs' in script.text:
                    duration_ms = script.text.split('approxDurationMs":"')[1].split('"')[0]
                    return int(duration_ms) / 1000
    return None

# Fungsi untuk mengonversi durasi ISO 8601 ke detik
def parse_duration_to_seconds(duration_iso):
    match = re.match(r'PT(\d+H)?(\d+M)?(\d+S)?', duration_iso)
    hours = int(match.group(1)[:-1]) if match.group(1) else 0
    minutes = int(match.group(2)[:-1]) if match.group(2) else 0
    seconds = int(match.group(3)[:-1]) if match.group(3) else 0
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

# URL file CSV
csv_url = "https://raw.githubusercontent.com/rii92/Datmin/main/topics.csv"

# Membaca data dari CSV
data = pd.read_csv(csv_url)

# Menambahkan kolom jumlah_detik ke data
data['jumlah_detik'] = 0

# Mengambil dan memperbarui durasi setiap video dalam detik
for index, row in data.iterrows():
    video_url = row['file']
    duration = get_video_duration(video_url)
    if duration:
        data.at[index, 'jumlah_detik'] = duration
    else:
        print(f"Tidak dapat mengambil durasi untuk URL: {video_url}")

# Menyimpan data yang diperbarui ke CSV baru
data.to_csv('updated_topics.csv', index=False)

print("Data telah diperbarui dan disimpan ke 'updated_topics.csv'")